In [98]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

In [99]:
def remove_unnamed(df):
    if "Unnamed: 0" in df.columns:
        del df["Unnamed: 0"]
    if "Unnamed: 0.1" in df.columns:
        del df["Unnamed: 0.1"]
    return df

In [100]:
source_water = "seawater"
df = pd.read_csv("data/baseline_cases_runs.csv")
cases = df[df.source_water_category == source_water].case_study.unique()
#cases = ["cherokee", "gila_river"]

In [101]:
os.makedirs("results/case_studies/%s" % source_water)
os.makedirs("results/case_studies/%s/baseline" % source_water)
os.makedirs("results/case_studies/%s/baseline_and_whatifs" % source_water)
os.makedirs("results/case_studies/%s/sensitivities" % source_water)
os.makedirs("results/case_studies/%s/baseline/figures" % source_water)
os.makedirs("results/case_studies/%s/baseline_and_whatifs/figures" % source_water)
os.makedirs("results/case_studies/%s/baseline/csvs" % source_water)
os.makedirs("results/case_studies/%s/baseline_and_whatifs/csvs" % source_water)
os.makedirs("results/case_studies/%s/baseline_and_whatifs/by_unit" % source_water)
os.makedirs("results/case_studies/%s/baseline/by_unit" % source_water)

In [102]:
# combine each case study what ifs

#combine each case study what ifs with baselines

In [103]:
for case in cases:
    case_wi_df = pd.DataFrame()
    path = './results/case_studies/'
    files = []
    for i in os.listdir(path):
        if os.path.isfile(os.path.join(path,i)) and case in i:
            if 'baseline' in i:
                continue
            if "whatifs" in i:
                continue
            else:
                print(case, i)
                files.append(i)
    
    for file in files:
        df = pd.read_csv('%s%s' % (path, file), index_col=False)
        case_wi_df = pd.concat([case_wi_df,df])
    
    #del case_wi_df["Unnamed: 0"]
    remove_unnamed(case_wi_df)
    #case_wi_df.to_csv("results/case_studies/%s/%s_whatifs.csv" % (source_water, case), index=False)
    
    # combining whatifs and baseline
    baseline_df = pd.read_csv('results/case_studies/%s_baseline.csv' % case, index_col=False)
    remove_unnamed(baseline_df)
    baseline_df.to_csv("results/case_studies/%s/baseline/%s_baseline.csv" % (source_water, case), index=False)
    baseline_df = pd.concat([baseline_df,case_wi_df])
    
    remove_unnamed(baseline_df)
    #del baseline_df["Unnamed: 0"]
    baseline_df.to_csv("results/case_studies/%s/baseline_and_whatifs/%s_baseline_and_whatifs.csv" % (source_water, 
                                                                                                     case), index=False)
    

ashkelon ashkelon_50%_fixed_onm_reduction.csv
ashkelon ashkelon_one_ro.csv
carlsbad carlsbad_one_ro.csv
carlsbad carlsbad_50%_fixed_onm_reduction.csv
santa_barbara santa_barbara_50%_fixed_onm_reduction.csv
tampa_bay tampa_bay_one_ro.csv
tampa_bay tampa_bay_50%_fixed_onm_reduction.csv


In [104]:
for file_type in ["baseline", "baseline_and_whatifs"]: #whatifs
    # combine baseline results and clean
    df = pd.DataFrame()

    if len(cases) == 2:
        df1 = pd.read_csv(("results/case_studies/%s/%s/%s_%s.csv" % (source_water, file_type, cases[0], file_type)), index_col=False)
        df2 = pd.read_csv(("results/case_studies/%s/%s/%s_%s.csv" % (source_water, file_type, cases[1], file_type)), index_col=False)
        df = pd.concat([df1, df2])
    if len(cases) == 3:
        df1 = pd.read_csv(("results/case_studies/%s/%s/%s_%s.csv" % (source_water, file_type, cases[0], file_type)), index_col=False)
        df2 = pd.read_csv(("results/case_studies/%s/%s/%s_%s.csv" % (source_water, file_type, cases[1], file_type)), index_col=False)
        df3 = pd.read_csv(("results/case_studies/%s/%s/%s_%s.csv" % (source_water, file_type, cases[2], file_type)), index_col=False)
        df = pd.concat([df1, df2, df3])
    if len(cases) == 4:
        df1 = pd.read_csv(("results/case_studies/%s/%s/%s_%s.csv" % (source_water, file_type, cases[0], file_type)), index_col=False)
        df2 = pd.read_csv(("results/case_studies/%s/%s/%s_%s.csv" % (source_water, file_type, cases[1], file_type)), index_col=False)
        df3 = pd.read_csv(("results/case_studies/%s/%s/%s_%s.csv" % (source_water, file_type, cases[2], file_type)), index_col=False)
        df4 = pd.read_csv(("results/case_studies/%s/%s/%s_%s.csv" % (source_water, file_type, cases[3], file_type)), index_col=False)
        df = pd.concat([df1, df2, df3, df4])

    df_names = pd.read_excel("data/Category_lookup.xlsx", sheet_name='Sheet1')
    df_names = df_names[df_names.CaseStudy.isin(cases)]
    df_names = df_names.set_index(df_names.caseunit)
    df["case_name"] = df["Case Study"] + "baseline" + df["Unit Process Name"] # WILL NEED TO EDIT FOR OTHER WHATIFS
    df["Treatment Category"] = df.case_name.map(df_names.category)
    remove_unnamed(df)
    
    df["Case Study"] = np.where(df["Case Study"] == "pulp_and_paper", "Pulp & Paper", df["Case Study"])
    df["Case Study"] = np.where(df["Case Study"] == "iron_and_steel", "Steel Plant", df["Case Study"])
    df["Case Study"] = np.where(df["Case Study"] == "upw", "Fab25 Wafer Fabrication", df["Case Study"])
    df["Case Study"] = np.where(df["Case Study"] == "ashkelon", "Ashkelon", df["Case Study"])
    df["Case Study"] = np.where(df["Case Study"] == "carlsbad", "Carlsbad", df["Case Study"])
    df["Case Study"] = np.where(df["Case Study"] == "santa_barbara", "Santa Barbara", df["Case Study"])
    df["Case Study"] = np.where(df["Case Study"] == "tampa_bay", "Tampa Bay", df["Case Study"])
    df["Case Study"] = np.where(df["Case Study"] == "big_spring", "Big Spring", df["Case Study"])
    df["Case Study"] = np.where(df["Case Study"] == "hrsd", "Hampton Roads", df["Case Study"])
    df["Case Study"] = np.where(df["Case Study"] == "ocwd", "Orange County", df["Case Study"])
    df["Case Study"] = np.where(df["Case Study"] == "solaire", "Solaire", df["Case Study"])

    df = df[df["Case Study"] != "Solaire"]

    # delete unwanted variables
    d_vars = ['Land', 'Working Capital', 'Fixed Capital Investment (FCI)', 
             'Employee Salaries', 'Benefits', 'Maintenance', 'Laboratory', 'Insurance and Taxes', 
             'Base Employee Salary', 'System Total Operating', 'System Fixed Operating',
             'System Other Variable Operating', 'System Total Operating', "Electricity Fraction of LCOW"]

    for d_var in d_vars:
        df = df[df.Variable != d_var]

    df = df.drop_duplicates()

    df.Variable = np.where(df.Variable == 'Other Operating', "Other Variable Operating", df.Variable)
    df.Variable = np.where(df.Variable == 'Total Fixed Operating ', 'Fixed Operating', df.Variable)
    remove_unnamed(df)
    df.to_csv("results/case_studies/%s/%s/%s_%s_results.csv" % (source_water, file_type, source_water,file_type))
    remove_unnamed(df)
    
    df["CaseStudy_Scenario"] = np.where(df.Scenario == "baseline", df["Case Study"],
                                        df["Case Study"] + " " + df.Scenario)
    ### Graphs
    
    sys_df = df[df["Unit Process Name"] == "System"]
    sys_df = sys_df[sys_df["Metric"] == "LCOW"]
    sys_df = sys_df[sys_df["Variable"] != "System Levelized Cost"]
    sys_df = sys_df[sys_df["Variable"] != "Electricity Fraction of LCOW"]
    sys_df = sys_df.sort_values("CaseStudy_Scenario", ascending=True)

    fig = px.bar(sys_df, x="CaseStudy_Scenario", y="Value", color="Variable",
                  labels={"Variable": "Portion of LCOW",  "Value": "Levelized Cost of Water ($/m3)"},
                  color_discrete_sequence=px.colors.qualitative.T10,
                template="simple_white")
    fig.show()
    fig.write_image("results/case_studies/%s/%s/figures/LCOW_Cost_Category.png" % (source_water,file_type),
                   format="png", engine="kaleido",
                   width=900, height=700, scale=1.7)
    
    

    for metric in ["Unit Levelized Cost", "Total Capital Investment (TCI)", "Annual O&M Costs", 
                  "Electricity Intensity System Treated"]:

        if metric == "Unit Levelized Cost": ylabel = "Levelized Cost of Water ($/m3)"
        if metric == "Total Capital Investment (TCI)": ylabel = "Total Capital Investment ($MM)"
        if metric == "Annual O&M Costs": ylabel = "Annual O&M Costs ($MM/yr)"
        if metric == "Electricity Intensity System Treated": ylabel = "Electricity Intensity kwh/m3 of Treated Water"

        sys_df = df[df["Unit Process Name"] != "System"]
        sys_df = sys_df[sys_df.Variable == metric]
        sys_df = sys_df.groupby(["CaseStudy_Scenario", "Treatment Category"]).sum()
        sys_df = sys_df.reset_index()
        sys_df = sys_df.sort_values("CaseStudy_Scenario", ascending=True)

        fig = px.bar(sys_df, x="CaseStudy_Scenario", y="Value", color="Treatment Category",
                      labels={"Variable": "",  "Value": ylabel},
                     color_discrete_sequence=px.colors.qualitative.T10,
                    template="simple_white")
        
        
        #fig.add_vrect(x0=0.9, x1=2)
        if metric == "Unit Levelized Cost": metric = "Treatment Category LCOW"
            
        fig.write_image("results/case_studies/%s/%s/figures/%s_%s.png" % (source_water,file_type, metric, file_type),
                                          format="png", engine="kaleido",
                   width=900, height=700, scale=1.7)
        sys_df.to_csv("results/case_studies/%s/%s/csvs/%s_%s.csv" % (source_water, file_type, metric, file_type))

        fig.show()


    for case_study in df["Case Study"].unique():

        sys_df = df[df["Case Study"] == case_study]

        for metric in ["Unit Levelized Cost", "Total Capital Investment (TCI)", "Annual O&M Costs", 
                      "Electricity Intensity System Treated"]:

            if metric == "Unit Levelized Cost": ylabel = "Levelized Cost of Water ($/m3)"
            if metric == "Total Capital Investment (TCI)": ylabel = "Total Capital Investment ($MM)"
            if metric == "Annual O&M Costs": ylabel = "Annual O&M Costs ($MM/yr)"
            if metric == "Electricity Intensity System Treated": ylabel = "Electricity Intensity kwh/m3 of Treated Water"

            sys_df1 = sys_df[sys_df["Unit Process Name"] != "System"]
            sys_df1 = sys_df1[sys_df1.Variable == metric]
            sys_df1 = sys_df1.groupby(["CaseStudy_Scenario", "Unit Process Name"]).sum()
            sys_df1 = sys_df1.reset_index()
            sys_df1 = sys_df1.sort_values("CaseStudy_Scenario", ascending=True)

            fig = px.bar(sys_df1, x="CaseStudy_Scenario", y="Value", color="Unit Process Name",
                          labels={"Variable": "",  "Value": ylabel},
                         color_discrete_sequence=px.colors.qualitative.Dark24,
                        template="simple_white")

            for data in fig.data:
                data["width"] = 0.5 #Change this value for bar widths
                
            fig.add_hline(y=0)
            
            fig.write_image("results/case_studies/%s/%s/by_unit/by_unit_%s_%s_%s.png" % (source_water,
                                               file_type, case_study,metric, 
                            file_type),format="png", engine="kaleido", width=900, height=700, scale=1.7)
                        
            sys_df.to_csv("results/case_studies/%s/%s/by_unit/by_unit_%s_%s_%s.csv" % (source_water,
                                  file_type, case_study,metric, file_type))

            fig.show()



    ### SUMMARY TABLE FOR THE SYSTEM ####   
    sys_df = df[df["Unit Process Name"] == "System"]

    metrics = ["System Total Capital Investment (TCI)",
    "System Catalysts and Chemicals",
    "System Electricity",
    "System Total Operating Cost",
    "System Levelized Cost",
    "Electricity Intensity",
    "Water Recovery"]

    sys_df = sys_df[sys_df.Variable.isin(metrics)]
    del sys_df["Unit Process Name"]; del sys_df["Treatment Category"]; 
    del sys_df["Metric"]; del sys_df["case_name"];
    sys_df.to_csv("results/case_studies/%s/%s/summary_table_%s_%s.csv" % (source_water, 
                                                            file_type, source_water, file_type), index=False)
    
    

In [ ]:
# Use `hole` to create a donut-like pie chart
fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.3)])
fig.show()

In [66]:
sys_df1

,CaseStudy_Scenario,Unit Process Name,Value
0,Tampa Bay,ammonia_addition,8.541628e-07
17,Tampa Bay,tri_media_filtration,2.654017e-01
16,Tampa Bay,treated_storage,0.000000e+00
15,Tampa Bay,sw_onshore_intake,1.372350e-01
14,Tampa Bay,surface_discharge,4.701202e-02
12,Tampa Bay,static_mixer,0.000000e+00
11,Tampa Bay,ro_second_pass,3.002904e-01
10,Tampa Bay,ro_first_pass,1.836085e+00
9,Tampa Bay,municipal_drinking,2.305778e-01
13,Tampa Bay,sulfuric_acid_addition,7.704731e-07


In [77]:
# Create subplots: use 'domain' type for Pie subplot
size_dict = []
for i in range(0, len(sys_df1.CaseStudy_Scenario.unique())):
    print(i)
    size_dict.append({'type':'domain'})

0
1
2


In [86]:
# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=1, cols=len(sys_df1.CaseStudy_Scenario.unique()), specs=[size_dict])
i = 1
for case_scenario in sys_df1.CaseStudy_Scenario.unique():
    dis_df = sys_df1[sys_df1.CaseStudy_Scenario == case_scenario]
    
    fig = go.Figure(data=[go.Pie(labels=dis_df["Unit Process Name"], values=dis_df.Value, hole=.3)])
    fig.update_layout(title_text=case_scenario)
        
    fig.show()

#     dis_df = sys_df1[sys_df1.CaseStudy_Scenario == case_scenario]
#     fig.add_trace(go.Pie(labels=dis_df["Unit Process Name"], values=dis_df.Value, name=case_scenario),1, i)
#     i = i +1
    
# Use `hole` to create a donut-like pie chart
#fig.update_traces(hole=.4, hoverinfo="label+percent+name")

# fig.update_layout(
#     title_text="Global Emissions 1990-2011",
#     # Add annotations in the center of the donut pies.
#     annotations=[dict(text='GHG', x=0.18, y=0.5, font_size=20, showarrow=False),
#                  dict(text='CO2', x=0.82, y=0.5, font_size=20, showarrow=False)])
#fig.show()

In [ ]:
            fig = px.bar(sys_df1, x="CaseStudy_Scenario", y="Value", color="Unit Process Name",
                          labels={"Variable": "",  "Value": ylabel},
                         color_discrete_sequence=px.colors.qualitative.Dark24,
                        template="simple_white")


In [ ]:
# combined baseline and what-ifs

In [ ]:
#### SENSE

In [105]:
all_sens_df = pd.DataFrame()

for source_water in ["seawater", "municipal", "brackish"]:

    df = pd.read_csv("data/baseline_cases_runs.csv")
    cases = df[df.source_water_category == source_water].case_study.unique()
    source_sens_df = pd.DataFrame()
    for case in cases:
    
        df1 = pd.read_csv("results/case_studies/%s_baseline_sensitivity.csv" % case)

        df1["source_water"] = np.array(source_water)
        df1["Case Study"] = np.array(case)
        
        source_sens_df = pd.concat([source_sens_df,df1])
        all_sens_df = pd.concat([all_sens_df,df1])
    
    source_sens_df.to_csv("results/case_studies/%s/sensitivities/%s_sensitivities.csv" % (source_water, source_water))
    
del all_sens_df["Unnamed: 0"]

all_sens_df.to_csv("results/case_studies/all_sens_df.csv")


FileNotFoundError: [Errno 2] No such file or directory: 'results/case_studies/municipal/sensitivities/municipal_sensitivities.csv'

In [24]:
all_sens_df.head()

,lcow,water_recovery,elec_lcow,elec_int,scenario_value,scenario_name,lcow_difference,water_recovery_difference,elec_lcow_difference,source_water,Case Study
0,0.418788,68.721316,41.580462,1.299507,NaN,baseline,-5.290213e-14,1.791900e-13,2.958744e-14,brackish,emwd
1,0.598268,68.721316,41.580462,1.299507,0.700,Plant Capacity Utilization 70-100%,1.794805e-01,9.992007e-16,5.551115e-17,brackish,emwd
2,0.585717,68.721316,41.580462,1.299507,0.715,Plant Capacity Utilization 70-100%,1.669294e-01,0.000000e+00,-5.551115e-17,brackish,emwd
3,0.573682,68.721316,41.580462,1.299507,0.730,Plant Capacity Utilization 70-100%,1.548941e-01,0.000000e+00,-5.551115e-17,brackish,emwd
4,0.562131,68.721316,41.580462,1.299507,0.745,Plant Capacity Utilization 70-100%,1.433435e-01,0.000000e+00,0.000000e+00,brackish,emwd
